In [1]:
## Final For testing

In [2]:
import pandas as pd
import numpy as np
import scipy
from nltk.corpus import stopwords
import re
import os

In [3]:
## Loading data - an excell file :

data1 = pd.ExcelFile('Tranzact2.xlsx')
tranzact_data1 = data1.parse(0)

lamba = len(tranzact_data1)

tranzact_data2 = tranzact_data1.iloc[0:lamba, 0:3] ## Remove unnecessary columns

tranzact_data2['Item'] = tranzact_data2['Item'].map(lambda x: x if type(x)!=str else x.lower()) ## make Items in lower case


tranzact_data3 = tranzact_data2.dropna() ## remove rows with cell value none

tranzact_data3 = tranzact_data3.reset_index(drop=True)

## Remove items sold by a company to its subsidiary :

drop_list = []

for i in range(len(tranzact_data3)):
    
    if (tranzact_data3['Supplier'][i].split())[0] == 'CESARE'and (tranzact_data3['Company'][i].split())[0] == 'CESARE':
        
        drop_list.append(i)
        
        tranzact_data = tranzact_data3.drop(drop_list)
                
tranzact_data = tranzact_data.reset_index(drop=True)

print(len(tranzact_data))
tranzact_data.head()       

24348


,Company,Supplier,Item
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
3,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
4,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."


In [4]:
## Cleaning the Item Description to create a feature matrix :

pd.options.mode.chained_assignment = None

tranzact_data['features'] = tranzact_data['Item'].str.replace(r'\b\d+\b','')  ## remove integers

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\W',' ')   ## remove puntuations

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\w\b','') ## remove stand alone single letters

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\s+',' ') ## remove gaps between words to singe gap

## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
words.append('min')
#words.append('machined')
words.append('top')
words.append('dia')
words.append('bar')

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

print(len(tranzact_data))
print('================')
print(tranzact_data['features'][111])
print(type(tranzact_data['features'][111]))

## Removing duplicate words from individual features

## tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
##                                                                                key=x.split().index))]).lower())
print(len(tranzact_data))

## Removing only single word features :

tranzact_data = tranzact_data[tranzact_data['features'].str.contains(' ')]
tranzact_data = tranzact_data.reset_index(drop=True)

tranzact_data.tail()

24348
chromium vanadium spring steel rolled en47 wire rod
<class 'str'>
24348


,Company,Supplier,Item,features
22325,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'r',ferrul number
22326,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
22327,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'b',ferrul number
22328,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'n',ferrul number
22329,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [5]:
for i in range(len(tranzact_data)):
    if tranzact_data['features'][i] == '':
        
        print(i)
        print('===')
        tranzact_data['Supplier'][i] = np.nan
    
tranzact_data = tranzact_data.dropna()
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))
tranzact_data.tail()

22330


,Company,Supplier,Item,features
22325,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'r',ferrul number
22326,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'y',ferrul number
22327,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'b',ferrul number
22328,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 4mm 'n',ferrul number
22329,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long


In [6]:
## Removing duplicate features + Supplier combinations :

tranzact_data['find_duplicate'] = tranzact_data['Supplier'].map(str) + tranzact_data['features']
tranzact_data.drop_duplicates(subset='find_duplicate', keep = 'first', inplace = True)
tranzact_data = tranzact_data.reset_index(drop=True)
print(len(tranzact_data))
tranzact_data.tail()

16192


,Company,Supplier,Item,features,find_duplicate
16187,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11,M/S. S. K. Enterprisepvc gland pg11
16188,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13,M/S. S. K. Enterprisepvc gland pg13
16189,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1,M/S. S. K. Enterprisepvc gland m25x1
16190,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 1mm '1',ferrul number,M/S. S. K. Enterpriseferrul number
16191,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long,Anant Tradershard copper pipe gauge long


In [7]:
## Removing unnecessary long features ( feature words > 25):

tranzact_data['number_of_words'] = tranzact_data.features.apply(lambda x: len(x.split()))

list_to_drop = []

for i in range(len(tranzact_data['number_of_words'])):
    
    if tranzact_data['number_of_words'][i] > 50 :
        
        list_to_drop.append(i)

tranzact_data_final = tranzact_data.drop(list_to_drop)
tranzact_data_final = tranzact_data_final.reset_index(drop=True)

print(max(tranzact_data_final['number_of_words']))

tranzact_data_final['word_increase'] = max(tranzact_data_final['number_of_words']) - tranzact_data_final['number_of_words']

tranzact_data_final.tail()

49


,Company,Supplier,Item,features,find_duplicate,number_of_words,word_increase
16177,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg11,pvc gland pg11,M/S. S. K. Enterprisepvc gland pg11,3,46
16178,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland pg13.5,pvc gland pg13,M/S. S. K. Enterprisepvc gland pg13,3,46
16179,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,pvc gland m25x1.5,pvc gland m25x1,M/S. S. K. Enterprisepvc gland m25x1,3,46
16180,Polfrost Air Con Pvt Ltd.,M/S. S. K. Enterprise,ferrul number 1mm '1',ferrul number,M/S. S. K. Enterpriseferrul number,2,47
16181,Airbex System Pvt. Ltd,Anant Traders,"7/8"" hard copper pipe 20 gauge 10' long",hard copper pipe gauge long,Anant Tradershard copper pipe gauge long,5,44


In [89]:
for i in range(len(tranzact_data_final)):
    
    dict_for_df = {}
    
#    print('no. of word increase = ', tranzact_data_final['word_increase'][i])
    
    if tranzact_data_final['word_increase'][i] > 0:
        
        for j in range(tranzact_data_final['word_increase'][i]):
            
            repeat_word = 'ww '
            dict_for_df[j] = [repeat_word]
        
        df = pd.DataFrame(dict_for_df)
        
        df['extra'] = df.sum(axis = 1)
        
    
    
        tranzact_data_final['features'][i] = df['extra'][0] + tranzact_data_final['features'][i]
        df.drop('extra', axis=1, inplace=True)
    
#    print('row no. = ', i)
#    print(tranzact_data_final['features'][i])
#    print('==================================================')
    
    
#    print(tranzact_data_final['features'][i])
#    print(len(tranzact_data_final['features'][i].split()))


In [5]:
## To check no. of Suppliers & no. of unique featuresin the data base :

# print('Supplier = ', len(tranzact_data.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data.features.value_counts()))
# print('Tranzact_data length = ', len(tranzact_data))
# tranzact_data.tail()

In [7]:
## Above indicates that there are 2337 suppliers supplying 28921 items(as per no. of rows)
## above indicates that there are 16728 unique features out of 28921 items(as per rows)

In [8]:
## Let's reduce no. of Suppliers : criterian - apperaing at least 10 times in the data frame

# tranzact_data_short = tranzact_data.groupby("Supplier").filter(lambda x: len(x) > 10)

# print(len(tranzact_data_short))
# print(len(tranzact_data_short.Supplier.value_counts()))

In [9]:
## Above indicates that there are now 391 suppliers supplying 23649 items(as per no. of rows)

In [8]:
## To cross check no. of Suppliers & no. of unique features in the data base :

# print('Supplier = ', len(tranzact_data_final.Supplier.value_counts()))
# print('Unique Features = ', len(tranzact_data_final.features.value_counts()))
# print('Total no. of entrys = ', len(tranzact_data_final))
# tranzact_data_final.tail()

In [8]:
# Creating seacrh engine file  and digitising original features :

from sklearn.feature_extraction.text import TfidfVectorizer

col = ['Supplier', 'features']
tranzact_data_search = tranzact_data_final[col].copy()


tranzact_data_search['Supplier_id'] = tranzact_data_search['Supplier'].factorize()[0]  ## Indexing

supplier_id_tranzact_data_search = tranzact_data_search[['Supplier', 'Supplier_id']].drop_duplicates(). \
                                                        sort_values('Supplier_id')

## Create two dictionaries :

supplier_to_id = dict(supplier_id_tranzact_data_search.values)
id_to_supplier = dict(supplier_id_tranzact_data_search[['Supplier_id', 'Supplier']].values)

## Digitising :

print(len(tranzact_data_search['features']))
print(len(tranzact_data_search['features'][0]))
print(tranzact_data_search['features'][0])

my_vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', ngram_range=(1, 2))
digital_features = my_vectorizer.fit(tranzact_data_search['features'])
my_vector = digital_features.transform(tranzact_data_search['features']).toarray()
print(my_vector.shape)
print(tranzact_data_search['features'][12490])
tranzact_data_search.tail()


16182
15
zincoplast zinc
(16182, 46713)
snuggle cups pink


,Supplier,features,Supplier_id
16177,M/S. S. K. Enterprise,pvc gland pg11,309
16178,M/S. S. K. Enterprise,pvc gland pg13,309
16179,M/S. S. K. Enterprise,pvc gland m25x1,309
16180,M/S. S. K. Enterprise,ferrul number,309
16181,Anant Traders,hard copper pipe gauge long,2055


In [9]:
## Input Text data :

text_data = ['head - weather proof [ip 67]']

#text_data_lower_case = [x.lower() for x in text_data]


#text_data1 = pd.Series(text_data_lower_case).apply(lambda x:sorted(set(x.split())))


print(text_data)
print(type(text_data))

['head - weather proof [ip 67]']
<class 'list'>


In [10]:
## From Teaxt data to text feature :

text_data_lower_case = [x.lower() for x in text_data]

text_data1 = pd.Series(text_data_lower_case)
#print(type(text_data1))
#print('==================')

#text_data1 = pd.Series(text_data1.apply(lambda x:sorted(set(x.split()))))
#print(type(text_data1))

pd.options.mode.chained_assignment = None

text_data_feature = text_data1.str.replace(r'\b\d+\b','')  ## remove integers

text_data_feature = text_data_feature.str.replace(r'\W',' ')   ## remove puntuations

text_data_feature = text_data_feature.str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

text_data_feature = text_data_feature.str.replace(r'\b\w\b','') ## remove stand alone single letters

text_data_feature = text_data_feature.str.replace(r'\s+',' ') ## remove gaps between words to singe gap


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
#words.append('machined')
words.append('top')
words.append('dia')

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

# text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
#                                                                                key=x.split().index))]).lower())

print(text_data_feature)
print("=========")
print(type(text_data_feature))
print('==================')
print(len(tranzact_data_search['features']))
## Cosine Similarity :

def cos_cdist(matrix, vector):
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

## Creating test vector from Text features :

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
test_digit_vector = []


merge_text_feature = tranzact_data_search['features'].append(pd.Series(text_data_feature))


print('==================')
print(len(tranzact_data_search['features']))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()

test_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)

print('%%%%%%%%%%%%%%%%%%%%%%%%%%')
print(merge_text_feature.shape)
print('===================')
print(merge_digit_vector.shape)
print('===================')
print(type(my_vector))
print(my_vector.shape)
print('===================')
print(type(test_digit_vector))
print(test_digit_vector.shape)
print('%%%%%%%%%%%%%%%%%%%%%%%%%')


## Finding cosine similarity :

c_d = []

# c_d = np.round((1 - cos_cdist(my_vector, test_digit_vector)), 3)
c_d = np.round((1 - cos_cdist(merge_digit_vector[0:len(merge_digit_vector)-2], test_digit_vector)), 3)
print(type(c_d))
print(c_d.shape)
print(text_data_feature)
print('+++++++++++')
print(len(tranzact_data_search))
#merge_text_feature.tail()

0    head weather proof ip
dtype: object
<class 'pandas.core.series.Series'>
16182
16182
%%%%%%%%%%%%%%%%%%%%%%%%%%
(16183,)
(16183, 46713)
<class 'numpy.ndarray'>
(16182, 46713)
<class 'numpy.ndarray'>
(46713,)
%%%%%%%%%%%%%%%%%%%%%%%%%
<class 'numpy.ndarray'>
(16181,)
0    head weather proof ip
dtype: object
+++++++++++
16182


In [11]:
result = []
result_frame = []
max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-50:][::-1]
#max_c_s_index = np.argsort(aa_sum)[-50:][::-1]

for i in max_c_s_index:
    
      
    result.append(np.array([c_d[i], tranzact_data_final['Item'][i], tranzact_data_search['features'][i], \
                            tranzact_data_search['Supplier'][i]]))

        
    
#    result.append(np.array([aa_sum[i], tranzact_data_final['Item'][i], tranzact_data_search['features'][i], \
#                            tranzact_data_search['Supplier'][i]]))


result_frame = pd.DataFrame(result, columns=['c_index', 'Item', 'features', 'Supplier'])


result_frame.c_index = result_frame.c_index.astype('float64')

result_frame.drop_duplicates(subset='Supplier', keep = 'first', inplace = True)
result_frame = result_frame.reset_index(drop=True)

kount = 0

for i in range(len(result_frame['Supplier'])):
    if result_frame['c_index'][i] >= 0.10:
        print(result_frame['Supplier'][i])
        print(i)
        kount = kount+1
        if kount == 5 :
            break
result_frame.head(10)

VARDHAMAN INDUSTRIES
0
ASCO NUMATICS (INDIA) PVT. LTD.
1
Emerson Process Management Chennai Pvt. Ltd
2
M/S. S. K. Enterprise
3
KAHAN CORPORATION
4


,c_index,Item,features,Supplier
0,0.757,"head - weather proof [ip 67] - 3/4"" et [se]",head weather proof ip et se,VARDHAMAN INDUSTRIES
1,0.205,"3/2way ,1/4” npt-f , ss316 body, nc, 2.4mm ori...",2way npt ss316 body nc orifice opr pressure we...,ASCO NUMATICS (INDIA) PVT. LTD.
2,0.184,"electro pneumatic smart(ff) positioner, make -...",electro pneumatic smart ff positioner make fis...,Emerson Process Management Chennai Pvt. Ltd
3,0.177,zetalux 100 x 100 x 70 ip 65,zetalux ip,M/S. S. K. Enterprise
4,0.122,ip 55 fan filter_elettro_jsav150/sf-4/e.g,ip fan filter_elettro_jsav150 sf,KAHAN CORPORATION
5,0.122,ip 55 fan filter_elettro_jsav150/sf-4/e.g,ip fan filter_elettro_jsav150 sf,YOGESH SWITCHGEARS & CABLES PVT.LTD.
6,0.113,polish paper water proof no.80,polish paper water proof,Kwality Auto Paints
7,0.109,150g water proof paper,150g water proof paper,Alpha Enterprises
8,0.103,hydraulic valve flame proof cover,hydraulic valve flame proof cover,ADITYA CONTROL SYSTEM
9,0.097,phosphor bronze proof machined casting,phosphor bronze proof machined casting,Ratnadeep Castings


In [62]:
print(result_frame['features'][1])
print(result_frame['features'][2])

2way npt ss316 body nc orifice opr pressure bar weather proof enclosure ip67 manual override class coil insulation
electro pneumatic smart ff positioner make fisher model dvc6200f foundation fieldbus weather proof electrical connection ip65 pneumatic connection


In [12]:
for j in range(10):
    
    if result_frame['features'][j] != result_frame['features'][j+1]:
        
                
        next_text_data = [result_frame['features'][j] + ' ' + result_frame['features'][j+1]]
                                        
        break
print(next_text_data)
print(type(next_text_data))

['head weather proof ip et se 2way npt ss316 body nc orifice opr pressure weather proof enclosure ip67 manual override class coil insulation']
<class 'list'>


In [13]:
## From Teaxt data to text feature :

text_data_lower_case = [x.lower() for x in next_text_data]

text_data1 = pd.Series(text_data_lower_case)
#print(type(text_data1))
#print('==================')

#text_data1 = pd.Series(text_data1.apply(lambda x:sorted(set(x.split()))))
#print(type(text_data1))

pd.options.mode.chained_assignment = None

text_data_feature = text_data1.str.replace(r'\b\d+\b','')  ## remove integers

text_data_feature = text_data_feature.str.replace(r'\W',' ')   ## remove puntuations

text_data_feature = text_data_feature.str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only

text_data_feature = text_data_feature.str.replace(r'\b\w\b','') ## remove stand alone single letters

text_data_feature = text_data_feature.str.replace(r'\s+',' ') ## remove gaps between words to singe gap


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")
words.remove('for')
words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')
words.append('max')
#words.append('machined')
words.append('top')
words.append('dia')

text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())

# text_data_feature = text_data_feature.apply(lambda x:' '.join([i for i in (sorted(set(x.split()), \
#                                                                                key=x.split().index))]).lower())

print(text_data_feature)
print("=========")
print(type(text_data_feature))
print('==================')
print(len(tranzact_data_search['features']))
## Cosine Similarity :

def cos_cdist(matrix, vector):
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

## Creating test vector from Text features :

merge_text_feature = []
merge_text_feature_digit = []
merge_digit_vector = []
test_digit_vector = []


merge_text_feature = tranzact_data_search['features'].append(pd.Series(text_data_feature))


print('==================')
print(len(tranzact_data_search['features']))

merge_text_feature = merge_text_feature.reset_index(drop=True)

merge_text_feature_digit = my_vectorizer.fit(merge_text_feature)

# encode document
merge_digit_vector = merge_text_feature_digit.transform(merge_text_feature).toarray()

test_digit_vector = merge_digit_vector[len(merge_digit_vector)-1].reshape((merge_digit_vector.shape)[1],)

print('%%%%%%%%%%%%%%%%%%%%%%%%%%')
print(merge_text_feature.shape)
print('===================')
print(merge_digit_vector.shape)
print('===================')
print(type(my_vector))
print(my_vector.shape)
print('===================')
print(type(test_digit_vector))
print(test_digit_vector.shape)
print('%%%%%%%%%%%%%%%%%%%%%%%%%')


## Finding cosine similarity :

c_d = []

# c_d = np.round((1 - cos_cdist(my_vector, test_digit_vector)), 3)
c_d = np.round((1 - cos_cdist(merge_digit_vector[0:len(merge_digit_vector)-2], test_digit_vector)), 3)
print(type(c_d))
print(c_d.shape)
print(text_data_feature)
print('+++++++++++')
print(len(tranzact_data_search))
#merge_text_feature.tail()

0    head weather proof ip et se 2way npt ss316 bod...
dtype: object
<class 'pandas.core.series.Series'>
16182
16182
%%%%%%%%%%%%%%%%%%%%%%%%%%
(16183,)
(16183, 46714)
<class 'numpy.ndarray'>
(16182, 46713)
<class 'numpy.ndarray'>
(46714,)
%%%%%%%%%%%%%%%%%%%%%%%%%
<class 'numpy.ndarray'>
(16181,)
0    head weather proof ip et se 2way npt ss316 bod...
dtype: object
+++++++++++
16182


In [14]:
result = []
result_frame = []
max_c_s_index = []
max_c_s_index = np.argsort(c_d)[-50:][::-1]
#max_c_s_index = np.argsort(aa_sum)[-50:][::-1]

for i in max_c_s_index:
    
      
    result.append(np.array([c_d[i], tranzact_data_final['Item'][i], tranzact_data_search['features'][i], \
                            tranzact_data_search['Supplier'][i]]))

        
    
#    result.append(np.array([aa_sum[i], tranzact_data_final['Item'][i], tranzact_data_search['features'][i], \
#                            tranzact_data_search['Supplier'][i]]))


result_frame = pd.DataFrame(result, columns=['c_index', 'Item', 'features', 'Supplier'])


result_frame.c_index = result_frame.c_index.astype('float64')

result_frame.drop_duplicates(subset='Supplier', keep = 'first', inplace = True)
result_frame = result_frame.reset_index(drop=True)

kount = 0

for i in range(len(result_frame['Supplier'])):
    if result_frame['c_index'][i] >= 0.10:
        print(result_frame['Supplier'][i])
        print(i)
        kount = kount+1
        if kount == 5 :
            break
result_frame.head(10)

ASCO NUMATICS (INDIA) PVT. LTD.
0
VARDHAMAN INDUSTRIES
1
IMI Norgren Herion Pvt.Ltd.
2
Entech Controls
3
Emerson Process Management Chennai Pvt. Ltd
4


,c_index,Item,features,Supplier
0,0.870,"3/2way ,1/4” npt-f , ss316 body, nc, 2.4mm ori...",2way npt ss316 body nc orifice opr pressure we...,ASCO NUMATICS (INDIA) PVT. LTD.
1,0.590,"head - weather proof [ip 67] - 3/4"" et [se]",head weather proof ip et se,VARDHAMAN INDUSTRIES
2,0.166,"3/2 way, 1/4""npt, direct acting universal, man...",way npt direct acting universal manual overrid...,IMI Norgren Herion Pvt.Ltd.
3,0.160,"solenoid valve,imi make, 3/2 way,1/4” npt-f, u...",solenoid valve imi make way npt universal auto...,Entech Controls
4,0.119,"electro pneumatic smart(ff) positioner, make -...",electro pneumatic smart ff positioner make fis...,Emerson Process Management Chennai Pvt. Ltd
5,0.116,"manual override, 200nm torque, moc: aluminium",manual override 200nm torque moc aluminium,Aira Euro Automation Pvt.Ltd.
6,0.113,"limit switch spdt, honeywell micro switch v15s...",limit switch spdt honeywell micro switch v15s0...,DEVSHRI MARKETING
7,0.111,"vent plug 1/2""npt(m) ss316",vent plug npt ss316,ATHARVA AGENCIES
8,0.095,"male elbow connector 3/8"" od x 1/4""npt (m) ss316",male elbow connector od npt ss316,VENUS ENGINEERS
9,0.094,pressure gauge for afr norgren:b38p-254-b1ma-i...,pressure gauge for afr norgren b38p b1ma ib9pe...,IMI NORGREN HERION PVT. LTD


In [17]:
print(result_frame['features'][2])

way npt direct acting universal manual override ss316l body orifice npt conduit coil wp ip coil
